In [7]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [38]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [39]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [40]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
392,2725348408,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
1095,4188937110,AK,"525 SECOND STREET, SKAGWAY, AK",40.670782,-73.976908,NaN
1601,9864858410,AK,"525 Second Street, Skagway, AK",40.670782,-73.976908,NaN
2874,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
3234,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
...,...,...,...,...,...,...
965547,9725917702,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965548,9666867710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965550,9724667708,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN


In [117]:
fips_check_scraped_null_tx = fips_check_scraped_null.loc[fips_check_scraped_null["BorrowerState"]=="TX"]
fips_check_scraped_null_tx

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
833008,6879468301,TX,"22452 US 77 Expy, Harlingen, TX",34.674350,-97.207957,NaN
835000,1259317204,TX,"22452 US 77 EXPY, HARLINGEN, TX",34.674350,-97.207957,NaN
837116,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN
837476,4722878309,TX,"1401 W US Highway 77, San Benito, TX",36.299336,-97.283176,NaN
840045,4657767002,TX,"9423 Highway 60 W, SNOOK, TX",36.216687,-98.770927,NaN
841458,4627727109,TX,"1401 West US HIGHWAY 77, SAN BENITO, TX",36.299336,-97.283176,NaN
844389,1047477106,TX,"US HIGHWAY 87, ADKINS, TX",38.455217,-104.606642,NaN
847600,7675668403,TX,"2513 Highway 75 S, Madisonville, TX",33.955462,-96.415656,NaN
848703,8780607004,TX,"100 I-35 North, WACO, TX",34.587650,-97.205132,NaN
850676,1552687100,TX,"24443 INTERSTATE 20 ACCESS RD, WILLS POINT, TX",32.487949,-91.976357,NaN


In [42]:
lat = fips_check_scraped_null_tx["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null_tx["Long"].astype(float).astype(str).to_list()

<h4>FIPS Script

In [13]:
#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [43]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            #startswith = [s for s in result_digits if s.startswith('51')] #VA
            fips_elements =[element for element in result_digits if len(element)==15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/24 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

Length Check

In [44]:
geo_id_list =geo_id 
print(len(geo_id_list))
print(geo_id_list)

Matching FIPS to DataFrame

In [203]:
fips_fill = fips_check_scraped_null_tx

In [204]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_18013/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
833008,6879468301,TX,"22452 US 77 Expy, Harlingen, TX",26.234425,-97.722283,[400496814001068]
835000,1259317204,TX,"22452 US 77 EXPY, HARLINGEN, TX",26.234425,-97.722283,[400496814001068]
837116,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.633582,-96.020147,[220730106061000]
837476,4722878309,TX,"1401 W US Highway 77, San Benito, TX",26.151359,-97.646059,[401039569001039]
840045,4657767002,TX,"9423 Highway 60 W, SNOOK, TX",30.492523,-96.478350,[400939552001281]
841458,4627727109,TX,"1401 West US HIGHWAY 77, SAN BENITO, TX",26.151359,-97.646059,[401039569001039]
844389,1047477106,TX,"US HIGHWAY 87, ADKINS, TX",29.373641,-98.272151,[081010029223023]
847600,7675668403,TX,"2513 Highway 75 S, Madisonville, TX",30.948402,-95.913418,[400137964001122]
848703,8780607004,TX,"100 I-35 North, WACO, TX",31.558433,-97.119712,[400496814001045]
850676,1552687100,TX,"24443 INTERSTATE 20 ACCESS RD, WILLS POINT, TX",32.615992,-95.974066,[220730106061000]


Remove brackets

In [205]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_18013/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
833008,6879468301,TX,"22452 US 77 Expy, Harlingen, TX",26.234425,-97.722283,400496814001068
835000,1259317204,TX,"22452 US 77 EXPY, HARLINGEN, TX",26.234425,-97.722283,400496814001068
837116,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.633582,-96.020147,220730106061000
837476,4722878309,TX,"1401 W US Highway 77, San Benito, TX",26.151359,-97.646059,401039569001039
840045,4657767002,TX,"9423 Highway 60 W, SNOOK, TX",30.492523,-96.478350,400939552001281
841458,4627727109,TX,"1401 West US HIGHWAY 77, SAN BENITO, TX",26.151359,-97.646059,401039569001039
844389,1047477106,TX,"US HIGHWAY 87, ADKINS, TX",29.373641,-98.272151,081010029223023
847600,7675668403,TX,"2513 Highway 75 S, Madisonville, TX",30.948402,-95.913418,400137964001122
848703,8780607004,TX,"100 I-35 North, WACO, TX",31.558433,-97.119712,400496814001045
850676,1552687100,TX,"24443 INTERSTATE 20 ACCESS RD, WILLS POINT, TX",32.615992,-95.974066,220730106061000


State Adjustment

In [50]:
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '05', 'BorrowerState'] = 'AR'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '08', 'BorrowerState'] = 'CO'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '09', 'BorrowerState'] = 'CT'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '11', 'BorrowerState'] = 'DC'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '12', 'BorrowerState'] = 'FL'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '20', 'BorrowerState'] = 'KS'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '22', 'BorrowerState'] = 'LA'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '24', 'BorrowerState'] = 'MD'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '35', 'BorrowerState'] = 'NM'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '36', 'BorrowerState'] = 'NY'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '37', 'BorrowerState'] = 'NC'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '40', 'BorrowerState'] = 'OK'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '47', 'BorrowerState'] = 'TN'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '48', 'BorrowerState'] = 'TX'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '50', 'BorrowerState'] = 'VT'
fips_fill.loc[fips_fill["FIPS_z"].str[:2] == '51', 'BorrowerState'] = 'VA'


In [54]:
print(fips_fill["LoanNumber"].values)

[6879468301 1259317204 5917947010 4722878309 4657767002 4627727109
 1047477106 7675668403 8780607004 1552687100 6694777101 3856588709
 4892627006 6483647010 3580538703 9393528310 5449747704 7852957010
 3325407706 2888408309 4997347709 1002917702 9693387104 5465077105]


Check

Manual Update

fips_fill.loc[fips_fill["LoanNumber"]==6879468301, "full_add"]=""
fips_fill.loc[fips_fill["LoanNumber"]==6879468301, "Lat"]=""
fips_fill.loc[fips_fill["LoanNumber"]==6879468301, "Long"]=""
fips_fill.loc[fips_fill["LoanNumber"]==6879468301, "FIPS_z"]=""
fips_fill.loc[fips_fill["LoanNumber"]==6879468301, "BorrowerState"]=""

In [240]:
data_xls = pd.read_excel('../../../data/null/tx_fips_update.xlsx', dtype=str, index_col=None)
data_xls.to_csv('../../../data/null/tx_fips_update.csv', encoding='utf-8', index=False)
tx_update_df = pd.read_csv("../../../data/null/tx_fips_update.csv").set_index(['index']).rename_axis(None)
tx_update_df.index = tx_update_df.index.astype('int64')
tx_update_df

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
833008,6879468301,TX,"22452 US 77 Expressway, Harlingen, TX",26.234425,-97.722283,480610102053001
835000,1259317204,TX,"22452 US 77 Expressway, Harlingen, TX",26.234425,-97.722283,480610102053001
837116,5917947010,TX,"28117 I-20 Frontage Rd, Wills Point, TX",32.633582,-96.020147,484679504004019
837476,4722878309,TX,"1401 US-77 BUS, San Benito, TX",26.151359,-97.646059,480610117011026
840045,4657767002,TX,"9500 Slovacek Rd, Snook, TX",30.492523,-96.478350,480519704001066
841458,4627727109,TX,"1401 US-77 BUS, San Benito, TX",26.151359,-97.646059,480610117011026
844389,1047477106,TX,"10865 US Hwy 87 E South, Adkins, TX",29.373641,-98.272151,480291318023023
847600,7675668403,TX,"101 W Trinity St, Madisonville, TX",30.948402,-95.913418,483130004001025
848703,8780607004,TX,"100 N Interstate 35 Frontage Rd, Waco, TX",31.558433,-97.119712,483090015001065
850676,1552687100,TX,"24443 Interstate 20 S Access Rd, Wills Point, TX",32.615992,-95.974066,484679506022011


In [241]:
tx_update_df.loc[tx_update_df["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Verify = #15 digits

In [180]:
fips_fill.loc[fips_fill["FIPS_z"].astype(str).str.len() > 15]

,LoanNumber,BorrowerState,full_Add,Lat,Long,FIPS_z


Read in base data

In [4]:
#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/ppp_geo_fips23.csv", index_col=0) 
fips_check.head()

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13


In [247]:
fips_check.loc[fips_check["LoanNumber"]==5917947010]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
837120,5917947010,TX,"28117 Interstate 20, WILLS POINT, TX",32.487949,-91.976357,NaN


Update

In [248]:
fips_check.loc[fips_check.LoanNumber.isin(tx_update_df.LoanNumber), ['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']] = tx_update_df[['BorrowerState', 'full_add', 'Lat', 'Long', 'FIPS_z']].values
fips_check

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5502308207,AK,"5015 BUSINESS PARK BLVD, ANCHORAGE, AK",61.174949,-149.888309,2.020002e+13
1,6110847106,AK,"7000 Uula St, BARROW, AK",71.297290,-156.728897,2.185000e+13
2,4539098204,AK,"4700 Old International Airport Rd, Anchorage, AK",61.173232,-149.972718,2.020002e+13
3,5120868804,AK,"540 W Intl Airport Rd, Anchorage, AK",61.173164,-149.893283,2.020003e+13
4,6650277102,AK,"4300 BARTLETT STREET, HOMER, AK",59.652704,-151.549793,2.122001e+13
...,...,...,...,...,...,...
965547,9725917702,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965548,9666867710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965549,9662697710,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN
965550,9724667708,Not Available,"17 S High St., COLUMBUS, OH",39.961717,-83.001011,NaN


Verify

In [5]:
fips_check.loc[fips_check["LoanNumber"]==1800157203]

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
884044,1800157203,TX,"995 RESEARCH BLVD STE 118, Austin, TX",30.47912,-97.802932,4.849102e+14


In [6]:
fips_check["BorrowerState"].value_counts()

BorrowerState
CA               129086
TX                76064
NY                73819
FL                60004
IL                40763
PA                38500
OH                32639
NJ                32515
MI                29791
MA                27179
GA                25787
WA                23620
VA                23217
NC                22798
MN                19720
CO                19668
MD                18954
WI                17346
IN                16594
MO                16339
TN                15864
AZ                15757
LA                13669
OR                13403
CT                12882
AL                10912
SC                10741
OK                 9995
UT                 9258
KY                 9178
IA                 8295
NV                 8254
KS                 8201
AR                 5857
NE                 5804
MS                 5504
NH                 5069
HI                 4979
NM                 4482
ID                 4415
DC                 4385
ME

Write to MainFrame

In [253]:
fips_check.to_csv("../../../data/state_data/geo/ppp_geo_fips23.csv") 

Run through fipssplit after write to update state Files